In [4]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import os
import json

# 自动下载模型时，指定使用modelscope; 否则，会从HuggingFace下载
os.environ['VLLM_USE_MODELSCOPE'] = 'True'

def get_completion(prompts, model, tokenizer=None, temperature=0.6, top_p=0.95, top_k=20, min_p=0, max_tokens=4096, max_model_len=8192):
    stop_token_ids = [151645, 151643]
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        min_p=min_p,
        max_tokens=max_tokens,
        stop_token_ids=stop_token_ids
    )
    llm = LLM(model=model, tokenizer=tokenizer, max_model_len=max_model_len,trust_remote_code=True)  # max_model_len 用于限制模型在推理过程中可以处理的最大输入和输出长度之和。
    outputs = llm.generate(prompts, sampling_params=sampling_params)
    return outputs

In [5]:
model = '/root/Qwen/Qwen/Qwen3-8B'
tokenzier = AutoTokenizer.from_pretrained(model, use_fast=True)

prompt = "给我一个关于大模型的简短介绍。"
messages = [
    {
        "role": "user",
        "content": prompt
    }
]
text = tokenzier.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking = True   # 是否开启思考模式
)

outputs = get_completion(text, model, tokenizer=None, temperature=0.6, top_p = 0.95, top_k=20, min_p=0)  # 对于思考模式，官方建议使用以下参数：temperature = 0.6，TopP = 0.95，TopK = 20，MinP = 0。

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, \nResponse: {generated_text!r}\n")

INFO 06-03 06:00:46 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-03 06:00:46 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-03 06:00:46 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-03 06:00:56 [config.py:793] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 06-03 06:00:56 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-03 06:00:58 [core.py:438] Waiting for init message from front-end.
INFO 06-03 06:00:58 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='/root/Qwen/Qwen/Qwen3-8B', speculative_config=None, tokenizer='/root/Qwen/Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_c

[W603 06:01:19.583397663 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W603 06:01:40.596964188 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[rank0]:[W603 06:01:40.622080964 ProcessGroupGloo.cpp:727] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


INFO 06-03 06:01:41 [parallel_state.py:1064] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
WARNING 06-03 06:01:41 [topk_topp_sampler.py:58] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
INFO 06-03 06:01:41 [gpu_model_runner.py:1531] Starting to load model /root/Qwen/Qwen/Qwen3-8B...
INFO 06-03 06:01:41 [cuda.py:217] Using Flash Attention backend on V1 engine.
ERROR 06-03 06:01:41 [core.py:500] EngineCore failed to start.
ERROR 06-03 06:01:41 [core.py:500] Traceback (most recent call last):
ERROR 06-03 06:01:41 [core.py:500]   File "/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 491, in run_engine_core
ERROR 06-03 06:01:41 [core.py:500]     engine_core = EngineCoreProc(*args, **kwargs)
ERROR 06-03 06:01:41 [core.py:500]   File "/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/vllm/v1/

Process EngineCore_0:
Traceback (most recent call last):
  File "/usr/local/miniconda3/envs/py310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/local/miniconda3/envs/py310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 504, in run_engine_core
    raise e
  File "/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 491, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
  File "/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 390, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 71, in __init__
    self.model_executor = executor_class(vllm_config)
  File "/usr/

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}